#               -- Mere analysis not explaination--

***This notebook doesn't explain the concepts used to answer one question that finds a mention forthwith. However, if the generous audience wants me to swell more on on the explaination, I would do that in a different notebook. So, please upvote and feel free to comment.***

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# *Question: Did the winning candidates of both REP and DEM parties win by a significant margin or was the margin much less?*

***To answer that question; I'll use Hypothesis Testing. As I said previously, I won't at all dwell on the explaination of different terminologies, however if one wants to know a lil more about Hypothesis Testing, please check my recent work:***

[Introduction to Hypothesis Testing and Estimation](http://www.kaggle.com/ritikpnayak/introduction-to-hypothesis-testing-and-estimation)

***Also, please feel free to ask me to make a new notebook explaining the much deeper concepts that the notebook implicitly espouses some information about***

In [ ]:
import random
import math

In [ ]:
df = pd.read_csv('/kaggle/input/us-election-2020/president_county_candidate.csv')

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df.candidate.value_counts()

In [ ]:
df.party.value_counts()

In [ ]:
dem = df[df['party'] == 'DEM']
rep = df[df['party'] == 'REP']

In [ ]:
dem.total_votes.value_counts().sort_index()

In [ ]:
df.info()

In [ ]:
won = df[df.won == True]
## if I write won = df[df.won == 'True']; it would show an empty df

In [ ]:
won.head()

In [ ]:
won.party.value_counts()

In [ ]:
won.total_votes.value_counts().sort_index()

***-- Skipping much of the EDA for there are a lot of notebooks that have already served the purpose --***

# Hypothesis: The winning candidates of both democratic and republican party have won by a significant (huge) margin

# Null Hypothesis: *(The exact opposite of the hypothesis that we are trying to examine)* The winning candidates of both democratic and republican party have won by a much lesser margin

***We have to employ a proof by contradiction sort of method. In this method, we will see whether our hypothesis is true or false by examining whether our null hypothesis is true or false. That is, if the null hypothesis is true, our hypothesis would be false, else otherwise.***

In [ ]:
class HypothesisTest(object):
    
    def __init__(self, data):
        self.data = data
        self.MakeModel()
        self.actual = self.TestStatistic(data)
        
    def PValue(self, iters = 1000):
        self.test_stats = [self.TestStatistic(self.RunModel()) for _ in range(iters)]
        count = sum(1 for x in self.test_stats if x >= self.actual)
        return count / iters
    
    def TestStatistic(self, data):
        raise UnimplementedMethodException()
        
    def MakeModel(self):
        pass
    
    def RunModel(self):
        raise UnimplementedMethodException()

In [ ]:
class DiffMeansPermute(HypothesisTest):
    
    def TestStatistic(self, data):
        group1, group2 = data
        test_stat = abs(np.mean(group1) - np.mean(group2))
        return test_stat
    
    def MakeModel(self):
        group1, group2 = self.data
        self.n, self.m = len(group1), len(group2)
        self.pool = np.hstack((group1, group2))
        
    def RunModel(self):
        np.random.shuffle(self.pool)
        data = self.pool[:self.n], self.pool[self.n:]
        return data

In [ ]:
data = random.sample(list(won[won.party == 'REP']['total_votes'].values), 1509), list(won[won.party == 'REP']['total_votes'].values)
ht = DiffMeansPermute(data)
pvalue = []
teststat = []
for _ in range(100):
    data = random.sample(list(won[won.party == 'REP']['total_votes'].values), 1509), list(won[won.party == 'REP']['total_votes'].values)
    ht = DiffMeansPermute(data)
    pvalue.append(ht.PValue())
    teststat.append(ht.TestStatistic(data))

In [ ]:
print('mean of pvalue: ', np.mean(pvalue))
print('median of pvalue: ', np.median(pvalue))
print('min of pvalue: ', np.min(pvalue))
print('max of pvalue: ', np.max(pvalue))
print('\n')
print('mean of teststat: ', np.mean(teststat))
print('median of teststat: ', np.median(teststat))
print('min of teststat: ', np.min(teststat))
print('max of teststat: ', np.max(teststat))

# Conclusion

***If the PValue is less (considering a threshold of 5%); it means that our null hypothesis is incorrect, else otherwise.***

***Now from the above code, as we can see that both the mean and the median is more than 65%, we shall conclude that our null hypothesis that "both the distributions are same (where DEM won and where REP won)" or put differently; "the winning candidates of both democratic and republican party have won by a much lesser margin" is true.***

***What is the margin?***

***One can look at the other values, of the 'teststatistic' list which shows that both the mean and the median of the margin (no. of votes with which a candidate won) is less than even 400 votes. Also the maximum margin is just over 1300 and the minimum margin being 10 ordinary votes.*** 